In [1]:
import os
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
from pandarallel import pandarallel
import multiprocessing as mp

In [2]:
#any microsim dir will work, just need to access the NHANES data
microsimDir = "/Users/deligkaris.1/OneDrive - The Ohio State University Wexner Medical Center/MICROSIM/CODE/microsim"
os.chdir(microsimDir)

In [3]:
import unittest
from microsim.person import Person
from microsim.initialization_repository import InitializationRepository
import numpy as np

from microsim.person import Person
from microsim.education import Education
from microsim.population import ClonePopulation
from microsim.test.test_risk_model_repository import TestRiskModelRepository
from microsim.bp_treatment_strategies import (
    AddBPTreatmentMedsToGoal120,
    AddASingleBPMedTreatmentStrategy,
    jnc8ForHighRiskLowBpTarget
)

In [57]:
class TestTreatmentStrategy(unittest.TestCase):
    def initializeAfib(person):
        return None

    def getPerson(self, baselineSBP=140, baselineDBP=80):
        return Person(
            age=75,
            gender=1,
            raceEthnicity=1,
            sbp=baselineSBP,
            dbp=baselineDBP,
            a1c=6.5,
            hdl=50,
            totChol=210,
            ldl=90,
            trig=150,
            bmi=22,
            waist=50,
            anyPhysicalActivity=0,
            education=Education.COLLEGEGRADUATE,
            smokingStatus=1,
            alcohol=0,
            antiHypertensiveCount=0,
            statin=0,
            otherLipidLoweringMedicationCount=0,
            creatinine=0.0,
            initializeAfib=TestTreatmentStrategy.initializeAfib,
            initializationRepository=InitializationRepository(),
            randomEffects={'gcp' : 0}
        )
    
    def testSprintStrategyAtPopLevelOneWave(self):
        maxMedPerson = self.getPerson(190, 120)
        pop = ClonePopulation(maxMedPerson, 100)
        pop._risk_model_repository = TestRiskModelRepository(nullModels=True)

        pop.set_bp_treatment_strategy(jnc8ForHighRiskLowBpTarget(0.075, {'sbp' : 126, 'dbp': 0}))
        alive, df = pop.advance_vectorized(1)

        # checking that anti hypertensives weren't accidentally added
        self.assertEqual(0, alive.antiHypertensiveCount.max())
        # should add 4 bp meds for everybody
        self.assertEqual(4, alive.bpMedsAdded.max())
        self.assertEqual(4, alive.bpMedsAdded.min())
        # should add 4 bp meds for everybody
        self.assertEqual(4, alive.totalBPMedsAdded.max())
        self.assertEqual(4, alive.totalBPMedsAdded.min())
        # BP should be lowered by 4*BP lowering effect
        self.assertEqual(190 - 4 * AddASingleBPMedTreatmentStrategy.sbpLowering, alive.sbp.max())
        self.assertEqual(190 - 4 * AddASingleBPMedTreatmentStrategy.sbpLowering, alive.sbp.min())

        for j in range(0,100):
            if pop._people.iloc[j]._age[-1]==75+1: #if person is alive, then check
                self.assertEqual(0, pop._people.iloc[j]._antiHypertensiveCount[-1])
                self.assertEqual(4, pop._people.iloc[j]._bpMedsAdded[-1])
                self.assertEqual(4, np.array(pop._people.iloc[j]._bpMedsAdded).sum())
                self.assertEqual(190 - 4 * AddASingleBPMedTreatmentStrategy.sbpLowering, pop._people.iloc[j]._sbp[-1])
        
    def testSprintStrategyAtPopLevelTwoWave(self):
        maxMedPerson = self.getPerson(190, 120)
        pop = ClonePopulation(maxMedPerson, 100)
        pop._risk_model_repository = TestRiskModelRepository(nullModels=True)
        pop.set_bp_treatment_strategy(jnc8ForHighRiskLowBpTarget(0.075, {'sbp' : 126, 'dbp': 0}))
        alive, df = pop.advance_vectorized(2)

        # checking that anti hypertensives weren't accidentally added
        self.assertEqual(0, alive.antiHypertensiveCount.max())
        # should add no new medicatinos
        self.assertEqual(0, alive.bpMedsAdded.max())
        self.assertEqual(0, alive.bpMedsAdded.min())
        # should add 4 bp meds for everybody
        self.assertEqual(4, alive.totalBPMedsAdded.max())
        self.assertEqual(4, alive.totalBPMedsAdded.min())
        # BP should be lowered by 4*BP lowering effect
        self.assertEqual(190 - 4 * AddASingleBPMedTreatmentStrategy.sbpLowering, alive.sbp.max())
        self.assertEqual(190 - 4 * AddASingleBPMedTreatmentStrategy.sbpLowering, alive.sbp.min())
        
        for j in range(0,100):
            if pop._people.iloc[j]._age[-1]==75+2: #if person is alive, then check
                self.assertEqual(0, pop._people.iloc[j]._antiHypertensiveCount[-1])
                self.assertEqual(0, pop._people.iloc[j]._bpMedsAdded[-1])
                self.assertEqual(4, np.array(pop._people.iloc[j]._bpMedsAdded).sum())
                self.assertEqual(190 - 4 * AddASingleBPMedTreatmentStrategy.sbpLowering, pop._people.iloc[j]._sbp[-1])
                break
                
    def testSprintStrategyAtPopLevelFiveWave(self):
        maxMedPerson = self.getPerson(190, 120)
        pop = ClonePopulation(maxMedPerson, 100)
        pop._risk_model_repository = TestRiskModelRepository(nullModels=True)
        pop.set_bp_treatment_strategy(jnc8ForHighRiskLowBpTarget(0.075, {'sbp' : 126, 'dbp': 0}))
        alive, df = pop.advance_vectorized(5)

        for j in range(0,100):
            if pop._people.iloc[j]._age[-1]==75+5: #if person is alive, then check
                for wave in range(2, 5):
                    self.assertEqual(0, pop._people.iloc[j]._antiHypertensiveCount[wave])
                    self.assertEqual(0, pop._people.iloc[j]._bpMedsAdded[wave])
                    self.assertEqual(4, np.array(pop._people.iloc[j]._bpMedsAdded).sum())
                    self.assertEqual(190 - 4 * AddASingleBPMedTreatmentStrategy.sbpLowering, pop._people.iloc[j]._sbp[wave])

        # checking that anti hypertensives weren't accidentally added
        self.assertEqual(0, alive.antiHypertensiveCount.max())
        # should add no new medicatinos
        self.assertEqual(0, alive.bpMedsAdded.max())
        self.assertEqual(0, alive.bpMedsAdded.min())
        # should add 4 bp meds for everybody
        self.assertEqual(4, alive.totalBPMedsAdded.max())
        self.assertEqual(4, alive.totalBPMedsAdded.min())
        # BP should be lowered by 4*BP lowering effect
        self.assertEqual(190 - 4 * AddASingleBPMedTreatmentStrategy.sbpLowering, alive.sbp.max())
        self.assertEqual(190 - 4 * AddASingleBPMedTreatmentStrategy.sbpLowering, alive.sbp.min())

In [58]:
test = TestTreatmentStrategy()

In [6]:
maxMedPerson = test.getPerson(190, 120)
pop = ClonePopulation(maxMedPerson, 100)
pop._risk_model_repository = TestRiskModelRepository(nullModels=True)
pop.set_bp_treatment_strategy(jnc8ForHighRiskLowBpTarget(0.075, {'sbp' : 126, 'dbp': 0}))
alive, df = pop.advance_vectorized(2)

# checking that anti hypertensives weren't accidentally added
0==alive.antiHypertensiveCount.max()


/Users/deligkaris.1/Library/CloudStorage/OneDrive-TheOhioStateUniversityWexnerMedicalCenter/MICROSIM/CODE/microsim/microsim/population.py:342: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  totalBPMedsAddedCapped.loc[totalBPMedsAddedCapped >= BaseTreatmentStrategy.MAX_BP_MEDS] = BaseTreatmentStrategy.MAX_BP_MEDS
/Users/deligkaris.1/Library/CloudStorage/OneDrive-TheOhioStateUniversityWexnerMedicalCenter/MICROSIM/CODE/microsim/microsim/gfr_equation.py:82: RuntimeWarning: divide by zero encountered in double_scalars
  return constant * (x.creatinine / crThreshold) ** exponent * 0.993**x.age
/Users/deligkaris.1/Library/CloudStorage/OneDrive-TheOhioStateUniversityWexnerMedicalCenter/MICROSIM/CODE/microsim/microsim/population.py:342: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice fr

True

In [7]:
# should add no new medicatinos
0==alive.bpMedsAdded.max()

True

In [8]:
0==alive.bpMedsAdded.min()

True

In [9]:
4==alive.totalBPMedsAdded.max()

True

In [10]:
4==alive.totalBPMedsAdded.min()

True

In [11]:
190 - 4 * AddASingleBPMedTreatmentStrategy.sbpLowering==alive.sbp.max()

True

In [12]:
190 - 4 * AddASingleBPMedTreatmentStrategy.sbpLowering==alive.sbp.min()

True

In [13]:
0==pop._people.iloc[0]._antiHypertensiveCount[-1]

True

In [14]:
0==pop._people.iloc[0]._bpMedsAdded[-1]

True

In [15]:
4==np.array(pop._people.iloc[0]._bpMedsAdded).sum()

True

In [16]:
190 - 4 * AddASingleBPMedTreatmentStrategy.sbpLowering==pop._people.iloc[0]._sbp[-1]

True

In [39]:
for i in range(0,1000):
    test = TestTreatmentStrategy()
    maxMedPerson = test.getPerson(190, 120)
    pop = ClonePopulation(maxMedPerson, 100)
    pop._risk_model_repository = TestRiskModelRepository(nullModels=True)
    pop.set_bp_treatment_strategy(jnc8ForHighRiskLowBpTarget(0.075, {'sbp' : 126, 'dbp': 0}))
    alive, df = pop.advance_vectorized(2)

    for j in range(0,100):
        if pop._people.iloc[j]._age[-1]==75+2:
            #print(0!=pop._people.iloc[0]._bpMedsAdded[-1])
            if ( (0!=alive.antiHypertensiveCount.max()) |
                 (0!=alive.bpMedsAdded.max()) |
                 (0!=alive.bpMedsAdded.min()) |
                 (4!=alive.totalBPMedsAdded.max()) |
                 (4!=alive.totalBPMedsAdded.min()) |
                 (190 - 4 * AddASingleBPMedTreatmentStrategy.sbpLowering!=alive.sbp.max()) |
                 (190 - 4 * AddASingleBPMedTreatmentStrategy.sbpLowering!=alive.sbp.min()) |
                 (0!=pop._people.iloc[0]._antiHypertensiveCount[-1]) |
                 (0!=pop._people.iloc[0]._bpMedsAdded[-1]) |
                 (4!=np.array(pop._people.iloc[0]._bpMedsAdded).sum()) |
                 (190 - 4 * AddASingleBPMedTreatmentStrategy.sbpLowering!=pop._people.iloc[0]._sbp[-1]) ) :
                print(i)
                break

/Users/deligkaris.1/Library/CloudStorage/OneDrive-TheOhioStateUniversityWexnerMedicalCenter/MICROSIM/CODE/microsim/microsim/population.py:342: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  totalBPMedsAddedCapped.loc[totalBPMedsAddedCapped >= BaseTreatmentStrategy.MAX_BP_MEDS] = BaseTreatmentStrategy.MAX_BP_MEDS
/Users/deligkaris.1/Library/CloudStorage/OneDrive-TheOhioStateUniversityWexnerMedicalCenter/MICROSIM/CODE/microsim/microsim/gfr_equation.py:82: RuntimeWarning: divide by zero encountered in double_scalars
  return constant * (x.creatinine / crThreshold) ** exponent * 0.993**x.age
/Users/deligkaris.1/Library/CloudStorage/OneDrive-TheOhioStateUniversityWexnerMedicalCenter/MICROSIM/CODE/microsim/microsim/population.py:342: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice fr

1


/Users/deligkaris.1/Library/CloudStorage/OneDrive-TheOhioStateUniversityWexnerMedicalCenter/MICROSIM/CODE/microsim/microsim/population.py:342: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  totalBPMedsAddedCapped.loc[totalBPMedsAddedCapped >= BaseTreatmentStrategy.MAX_BP_MEDS] = BaseTreatmentStrategy.MAX_BP_MEDS
/Users/deligkaris.1/Library/CloudStorage/OneDrive-TheOhioStateUniversityWexnerMedicalCenter/MICROSIM/CODE/microsim/microsim/gfr_equation.py:82: RuntimeWarning: divide by zero encountered in double_scalars
  return constant * (x.creatinine / crThreshold) ** exponent * 0.993**x.age
/Users/deligkaris.1/Library/CloudStorage/OneDrive-TheOhioStateUniversityWexnerMedicalCenter/MICROSIM/CODE/microsim/microsim/population.py:342: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice fr

/Users/deligkaris.1/Library/CloudStorage/OneDrive-TheOhioStateUniversityWexnerMedicalCenter/MICROSIM/CODE/microsim/microsim/population.py:342: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  totalBPMedsAddedCapped.loc[totalBPMedsAddedCapped >= BaseTreatmentStrategy.MAX_BP_MEDS] = BaseTreatmentStrategy.MAX_BP_MEDS
/Users/deligkaris.1/Library/CloudStorage/OneDrive-TheOhioStateUniversityWexnerMedicalCenter/MICROSIM/CODE/microsim/microsim/gfr_equation.py:82: RuntimeWarning: divide by zero encountered in double_scalars
  return constant * (x.creatinine / crThreshold) ** exponent * 0.993**x.age
/Users/deligkaris.1/Library/CloudStorage/OneDrive-TheOhioStateUniversityWexnerMedicalCenter/MICROSIM/CODE/microsim/microsim/population.py:342: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice fr

/Users/deligkaris.1/Library/CloudStorage/OneDrive-TheOhioStateUniversityWexnerMedicalCenter/MICROSIM/CODE/microsim/microsim/population.py:342: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  totalBPMedsAddedCapped.loc[totalBPMedsAddedCapped >= BaseTreatmentStrategy.MAX_BP_MEDS] = BaseTreatmentStrategy.MAX_BP_MEDS
/Users/deligkaris.1/Library/CloudStorage/OneDrive-TheOhioStateUniversityWexnerMedicalCenter/MICROSIM/CODE/microsim/microsim/gfr_equation.py:82: RuntimeWarning: divide by zero encountered in double_scalars
  return constant * (x.creatinine / crThreshold) ** exponent * 0.993**x.age
/Users/deligkaris.1/Library/CloudStorage/OneDrive-TheOhioStateUniversityWexnerMedicalCenter/MICROSIM/CODE/microsim/microsim/population.py:342: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice fr

...

744


/Users/deligkaris.1/Library/CloudStorage/OneDrive-TheOhioStateUniversityWexnerMedicalCenter/MICROSIM/CODE/microsim/microsim/population.py:342: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  totalBPMedsAddedCapped.loc[totalBPMedsAddedCapped >= BaseTreatmentStrategy.MAX_BP_MEDS] = BaseTreatmentStrategy.MAX_BP_MEDS
/Users/deligkaris.1/Library/CloudStorage/OneDrive-TheOhioStateUniversityWexnerMedicalCenter/MICROSIM/CODE/microsim/microsim/gfr_equation.py:82: RuntimeWarning: divide by zero encountered in double_scalars
  return constant * (x.creatinine / crThreshold) ** exponent * 0.993**x.age
/Users/deligkaris.1/Library/CloudStorage/OneDrive-TheOhioStateUniversityWexnerMedicalCenter/MICROSIM/CODE/microsim/microsim/population.py:342: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice fr

/Users/deligkaris.1/Library/CloudStorage/OneDrive-TheOhioStateUniversityWexnerMedicalCenter/MICROSIM/CODE/microsim/microsim/gfr_equation.py:82: RuntimeWarning: divide by zero encountered in double_scalars
  return constant * (x.creatinine / crThreshold) ** exponent * 0.993**x.age
/Users/deligkaris.1/Library/CloudStorage/OneDrive-TheOhioStateUniversityWexnerMedicalCenter/MICROSIM/CODE/microsim/microsim/population.py:342: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  totalBPMedsAddedCapped.loc[totalBPMedsAddedCapped >= BaseTreatmentStrategy.MAX_BP_MEDS] = BaseTreatmentStrategy.MAX_BP_MEDS
/Users/deligkaris.1/Library/CloudStorage/OneDrive-TheOhioStateUniversityWexnerMedicalCenter/MICROSIM/CODE/microsim/microsim/gfr_equation.py:82: RuntimeWarning: divide by zero encountered in double_scalars
  return const

/Users/deligkaris.1/Library/CloudStorage/OneDrive-TheOhioStateUniversityWexnerMedicalCenter/MICROSIM/CODE/microsim/microsim/population.py:342: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  totalBPMedsAddedCapped.loc[totalBPMedsAddedCapped >= BaseTreatmentStrategy.MAX_BP_MEDS] = BaseTreatmentStrategy.MAX_BP_MEDS
/Users/deligkaris.1/Library/CloudStorage/OneDrive-TheOhioStateUniversityWexnerMedicalCenter/MICROSIM/CODE/microsim/microsim/gfr_equation.py:82: RuntimeWarning: divide by zero encountered in double_scalars
  return constant * (x.creatinine / crThreshold) ** exponent * 0.993**x.age
/Users/deligkaris.1/Library/CloudStorage/OneDrive-TheOhioStateUniversityWexnerMedicalCenter/MICROSIM/CODE/microsim/microsim/population.py:342: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice fr

KeyboardInterrupt: 

In [40]:
i

760

In [21]:
0!=pop._people.iloc[0]._bpMedsAdded[-1]

True

In [25]:
pop._people.iloc[0]._bpMedsAdded[-1]

4

In [27]:
for i in range(0,100):
    print(pop._people.iloc[i]._bpMedsAdded)

[0, 4]
[0, 4, 0]
[0, 4, 0]
[0, 4, 0]
[0, 4, 0]
[0, 4, 0]
[0, 4, 0]
[0, 4, 0]
[0, 4]
[0, 4, 0]
[0, 4, 0]
[0, 4, 0]
[0, 4, 0]
[0, 4, 0]
[0, 4, 0]
[0, 4, 0]
[0, 4, 0]
[0, 4, 0]
[0, 4, 0]
[0, 4, 0]
[0, 4, 0]
[0, 4, 0]
[0, 4, 0]
[0, 4, 0]
[0, 4, 0]
[0, 4, 0]
[0, 4, 0]
[0, 4, 0]
[0, 4, 0]
[0, 4, 0]
[0, 4, 0]
[0, 4, 0]
[0, 4, 0]
[0, 4, 0]
[0, 4, 0]
[0, 4, 0]
[0, 4, 0]
[0, 4, 0]
[0, 4, 0]
[0, 4, 0]
[0, 4, 0]
[0, 4, 0]
[0, 4, 0]
[0, 4, 0]
[0, 4]
[0, 4, 0]
[0, 4, 0]
[0, 4, 0]
[0, 4, 0]
[0, 4, 0]
[0, 4, 0]
[0, 4, 0]
[0, 4, 0]
[0, 4, 0]
[0, 4, 0]
[0, 4, 0]
[0, 4, 0]
[0, 4, 0]
[0, 4, 0]
[0, 4, 0]
[0, 4, 0]
[0, 4, 0]
[0, 4, 0]
[0, 4, 0]
[0, 4, 0]
[0, 4]
[0, 4, 0]
[0, 4, 0]
[0, 4, 0]
[0, 4, 0]
[0, 4, 0]
[0, 4, 0]
[0, 4, 0]
[0, 4, 0]
[0, 4, 0]
[0, 4, 0]
[0, 4, 0]
[0, 4, 0]
[0, 4, 0]
[0, 4, 0]
[0, 4, 0]
[0, 4, 0]
[0, 4, 0]
[0, 4]
[0, 4]
[0, 4, 0]
[0, 4, 0]
[0, 4, 0]
[0, 4, 0]
[0, 4, 0]
[0, 4, 0]
[0, 4, 0]
[0, 4, 0]
[0, 4, 0]
[0, 4, 0]
[0, 4, 0]
[0, 4, 0]
[0, 4, 0]
[0, 4, 0]
[0, 4, 0]


In [35]:
pop._people.iloc[0], pop._people.iloc[0]._age[-1]

(Person(age=75.0, gender=1, race/eth=1, sbp=168.0, dbp=107.6, a1c=6.5, hdl=50.0, totChol=210.0, bmi=22.0, ldl=90.0, trig=150.0, smoking=1, waist=50.0, anyPhysicalActivity=0.0, alcohol=0, education=5, antiHypertensiveCount=0.0, otherLipid=0.0, creatinine=0.0, statin=0.0, index=0, outcomes={<OutcomeType.MI: 'mi'>: [], <OutcomeType.STROKE: 'stroke'>: [], <OutcomeType.DEMENTIA: 'dementia'>: []}),
 75.0)

In [34]:
pop._people.iloc[1], pop._people.iloc[1]._age

(Person(age=77.0, gender=1, race/eth=1, sbp=168.0, dbp=107.6, a1c=6.5, hdl=50.0, totChol=210.0, bmi=22.0, ldl=90.0, trig=150.0, smoking=1, waist=50.0, anyPhysicalActivity=0.0, alcohol=0, education=5, antiHypertensiveCount=0.0, otherLipid=0.0, creatinine=0.0, statin=0.0, index=1, outcomes={<OutcomeType.MI: 'mi'>: [], <OutcomeType.STROKE: 'stroke'>: [], <OutcomeType.DEMENTIA: 'dementia'>: []}),
 [75.0, 76.0, 77.0])

In [38]:
for j in range(0,100):
    print(j)
    if pop._people.iloc[j]._age[-1]==75+2:
        print(0!=pop._people.iloc[j]._bpMedsAdded[-1])
        break

0
1
False


In [29]:
alive

,age,baseAge,gender,raceEthnicity,black,sbp,dbp,a1c,current_diabetes,gfr,hdl,ldl,trig,totChol,bmi,anyPhysicalActivity,education,afib,alcoholPerWeek,creatinine,antiHypertensiveCount,current_bp_treatment,statin,otherLipidLoweringMedicationCount,waist,smokingStatus,current_smoker,dead,gcpRandomEffect,miPriorToSim,mi,stroke,ageAtFirstStroke,ageAtFirstMI,ageAtFirstDementia,miInSim,strokePriorToSim,strokeInSim,dementia,gcp,baseGcp,totalYearsInSim,totalQalys,totalBPMedsAdded,bpMedsAdded,populationIndex,sbp0,dbp0,a1c0,hdl0,ldl0,trig0,totChol0,bmi0,anyPhysicalActivity0,afib0,waist0,alcoholPerWeek0,creatinine0,age0,antiHypertensiveCount0,statin0,bpMedsAdded0,meanSbp,meanDbp,meanA1c,meanHdl,meanLdl,meanTrig,meanTotchol,meanBmi,meanAnyphysicalactivity,meanAfib,meanWaist,meanAlcoholperweek,meanCreatinine,meanAge,meanAntihypertensivecount,meanStatin,meanBpmedsadded,sbp1,dbp1,a1c1,hdl1,ldl1,trig1,totChol1,bmi1,anyPhysicalActivity1,afib1,waist1,alcoholPerWeek1,creatinine1,antiHypertensiveCount1,statin1,mi1,stroke1,dead1,gcpSlope,sbp2,dbp2,a1c2,hdl2,ldl2,trig2,totChol2,bmi2,anyPhysicalActivity2,afib2,waist2,alcoholPerWeek2,creatinine2,antiHypertensiveCount2,statin2,mi2,stroke2,dead2
0,77.0,75.0,1,1,False,168.0,107.6,6.5,False,inf,50.0,90.0,150.0,210.0,22.0,0.0,5,None,0,0.0,0.0,False,0.0,0.0,50.0,1,False,False,0,0,False,False,NaN,NaN,None,False,0,False,False,50.501099,55.327749,2,1.8900,4,0,1,190.0,120.0,6.5,50.0,90.0,150.0,210.0,22.0,0.0,None,50.0,0,0.0,75.0,0.0,0.0,0,175.333333,111.733333,6.5,50.0,90.0,150.0,210.0,22.0,0.0,NaN,50.0,0.0,0.0,75.0,0.0,0.0,0.0,168.0,107.6,6.5,50.0,90.0,150.0,210.0,22.0,0.0,None,50.0,0,0.0,0.0,0.0,False,False,False,-2.842979,168.0,107.6,6.5,50.0,90.0,150.0,210.0,22.0,0.0,None,50.0,0,0.0,0.0,0.0,False,False,False
1,77.0,75.0,1,1,False,168.0,107.6,6.5,False,inf,50.0,90.0,150.0,210.0,22.0,0.0,5,None,0,0.0,0.0,False,0.0,0.0,50.0,1,False,False,0,0,False,False,NaN,NaN,None,False,0,False,False,48.660266,55.327749,2,1.8900,4,0,2,190.0,120.0,6.5,50.0,90.0,150.0,210.0,22.0,0.0,None,50.0,0,0.0,75.0,0.0,0.0,0,175.333333,111.733333,6.5,50.0,90.0,150.0,210.0,22.0,0.0,NaN,50.0,0.0,0.0,75.0,0.0,0.0,0.0,168.0,107.6,6.5,50.0,90.0,150.0,210.0,22.0,0.0,None,50.0,0,0.0,0.0,0.0,False,False,False,-2.358973,168.0,107.6,6.5,50.0,90.0,150.0,210.0,22.0,0.0,None,50.0,0,0.0,0.0,0.0,False,False,False
2,77.0,75.0,1,1,False,168.0,107.6,6.5,False,inf,50.0,90.0,150.0,210.0,22.0,0.0,5,None,0,0.0,0.0,False,0.0,0.0,50.0,1,False,False,0,0,False,False,NaN,NaN,None,False,0,False,False,49.710026,55.327749,2,1.8900,4,0,3,190.0,120.0,6.5,50.0,90.0,150.0,210.0,22.0,0.0,None,50.0,0,0.0,75.0,0.0,0.0,0,175.333333,111.733333,6.5,50.0,90.0,150.0,210.0,22.0,0.0,NaN,50.0,0.0,0.0,75.0,0.0,0.0,0.0,168.0,107.6,6.5,50.0,90.0,150.0,210.0,22.0,0.0,None,50.0,0,0.0,0.0,0.0,False,False,False,-8.442398,168.0,107.6,6.5,50.0,90.0,150.0,210.0,22.0,0.0,None,50.0,0,0.0,0.0,0.0,False,False,False
3,77.0,75.0,1,1,False,168.0,107.6,6.5,False,inf,50.0,90.0,150.0,210.0,22.0,0.0,5,None,0,0.0,0.0,False,0.0,0.0,50.0,1,False,False,0,0,False,False,NaN,NaN,None,False,0,False,False,44.038474,55.327749,2,1.8900,4,0,4,190.0,120.0,6.5,50.0,90.0,150.0,210.0,22.0,0.0,None,50.0,0,0.0,75.0,0.0,0.0,0,175.333333,111.733333,6.5,50.0,90.0,150.0,210.0,22.0,0.0,NaN,50.0,0.0,0.0,75.0,0.0,0.0,0.0,168.0,107.6,6.5,50.0,90.0,150.0,210.0,22.0,0.0,None,50.0,0,0.0,0.0,0.0,False,False,False,-17.267429,168.0,107.6,6.5,50.0,90.0,150.0,210.0,22.0,0.0,None,50.0,0,0.0,0.0,0.0,False,False,False
4,77.0,75.0,1,1,False,168.0,107.6,6.5,False,inf,50.0,90.0,150.0,210.0,22.0,0.0,5,None,0,0.0,0.0,False,0.0,0.0,50.0,1,False,False,0,0,False,False,NaN,NaN,None,False,0,False,False,55.940935,55.327749,2,1.8900,4,0,5,190.0,120.0,6.5,50.0,90.0,150.0,210.0,22.0,0.0,None,50.0,0,0.0,75.0,0.0,0.0,0,175.333333,111.733333,6.5,50.0,90.0,150.0,210.0,22.0,0.0,NaN,50.0,0.0,0.0,75.0,0.0,0.0,0.0,168.0,107.6,6.5,50.0,90.0,150.0,210.0,22.0,0.0,None,50.0,0,0.0,0.0,0.0,False,False,False,-4.386881,168.0,107.6,6.5,50.0,90.0,150.0,210.0,22.0,0.0,None,5

In [31]:
df

,age,baseAge,gender,raceEthnicity,black,sbp,dbp,a1c,current_diabetes,gfr,hdl,ldl,trig,totChol,bmi,anyPhysicalActivity,education,afib,alcoholPerWeek,creatinine,antiHypertensiveCount,current_bp_treatment,statin,otherLipidLoweringMedicationCount,waist,smokingStatus,current_smoker,dead,gcpRandomEffect,miPriorToSim,mi,stroke,ageAtFirstStroke,ageAtFirstMI,ageAtFirstDementia,miInSim,strokePriorToSim,strokeInSim,dementia,gcp,baseGcp,gcpSlope,totalYearsInSim,totalQalys,totalBPMedsAdded,bpMedsAdded,populationIndex,sbp0,dbp0,a1c0,hdl0,ldl0,trig0,totChol0,bmi0,anyPhysicalActivity0,afib0,waist0,alcoholPerWeek0,creatinine0,age0,antiHypertensiveCount0,statin0,bpMedsAdded0,meanSbp,meanDbp,meanA1c,meanHdl,meanLdl,meanTrig,meanTotchol,meanBmi,meanAnyphysicalactivity,meanAfib,meanWaist,meanAlcoholperweek,meanCreatinine,meanAge,meanAntihypertensivecount,meanStatin,meanBpmedsadded
0,75.0,75.0,1,1,False,190.0,120.0,6.5,False,inf,50.0,90.0,150.0,210.0,22.0,0.0,5,None,0,0.0,0.0,False,0.0,0.0,50.0,1,False,False,0,0,False,False,None,None,None,False,0,False,False,55.327749,55.327749,0,0,0.0,0,0,0,190.0,120.0,6.5,50.0,90.0,150.0,210.0,22.0,0.0,None,50.0,0,0.0,75.0,0.0,0.0,0,190.0,120.0,6.5,50.0,90.0,150.0,210.0,22.0,0.0,NaN,50.0,0.0,0.0,75.0,0.0,0.0,0.0
1,75.0,75.0,1,1,False,190.0,120.0,6.5,False,inf,50.0,90.0,150.0,210.0,22.0,0.0,5,None,0,0.0,0.0,False,0.0,0.0,50.0,1,False,False,0,0,False,False,None,None,None,False,0,False,False,55.327749,55.327749,0,0,0.0,0,0,1,190.0,120.0,6.5,50.0,90.0,150.0,210.0,22.0,0.0,None,50.0,0,0.0,75.0,0.0,0.0,0,190.0,120.0,6.5,50.0,90.0,150.0,210.0,22.0,0.0,NaN,50.0,0.0,0.0,75.0,0.0,0.0,0.0
2,75.0,75.0,1,1,False,190.0,120.0,6.5,False,inf,50.0,90.0,150.0,210.0,22.0,0.0,5,None,0,0.0,0.0,False,0.0,0.0,50.0,1,False,False,0,0,False,False,None,None,None,False,0,False,False,55.327749,55.327749,0,0,0.0,0,0,2,190.0,120.0,6.5,50.0,90.0,150.0,210.0,22.0,0.0,None,50.0,0,0.0,75.0,0.0,0.0,0,190.0,120.0,6.5,50.0,90.0,150.0,210.0,22.0,0.0,NaN,50.0,0.0,0.0,75.0,0.0,0.0,0.0
3,75.0,75.0,1,1,False,190.0,120.0,6.5,False,inf,50.0,90.0,150.0,210.0,22.0,0.0,5,None,0,0.0,0.0,False,0.0,0.0,50.0,1,False,False,0,0,False,False,None,None,None,False,0,False,False,55.327749,55.327749,0,0,0.0,0,0,3,190.0,120.0,6.5,50.0,90.0,150.0,210.0,22.0,0.0,None,50.0,0,0.0,75.0,0.0,0.0,0,190.0,120.0,6.5,50.0,90.0,150.0,210.0,22.0,0.0,NaN,50.0,0.0,0.0,75.0,0.0,0.0,0.0
4,75.0,75.0,1,1,False,190.0,120.0,6.5,False,inf,50.0,90.0,150.0,210.0,22.0,0.0,5,None,0,0.0,0.0,False,0.0,0.0,50.0,1,False,False,0,0,False,False,None,None,None,False,0,False,False,55.327749,55.327749,0,0,0.0,0,0,4,190.0,120.0,6.5,50.0,90.0,150.0,210.0,22.0,0.0,None,50.0,0,0.0,75.0,0.0,0.0,0,190.0,120.0,6.5,50.0,90.0,150.0,210.0,22.0,0.0,NaN,50.0,0.0,0.0,75.0,0.0,0.0,0.0
5,75.0,75.0,1,1,False,190.0,120.0,6.5,False,inf,50.0,90.0,150.0,210.0,22.0,0.0,5,None,0,0.0,0.0,False,0.0,0.0,50.0,1,False,False,0,0,False,False,None,None,None,False,0,False,False,55.327749,55.327749,0,0,0.0,0,0,5,190.0,120.0,6.5,50.0,90.0,150.0,210.0,22.0,0.0,None,50.0,0,0.0,75.0,0.0,0.0,0,190.0,120.0,6.5,50.0,90.0,150.0,210.0,22.0,0.0,NaN,50.0,0.0,0.0,75.0,0.0,0.0,0.0
6,75.0,75.0,1,1,False,190.0,120.0,6.5,False,inf,50.0,90.0,150.0,210.0,22.0,0.0,5,None,0,0.0,0.0,False,0.0,0.0,50.0,1,False,False,0,0,False,False,None,None,None,False,0,False,False,55.327749,55.327749,0,0,0.0,0,0,6,190.0,120.0,6.5,50.0,90.0,150.0,210.0,22.0,0.0,None,50.0,0,0.0,75.0,0.0,0.0,0,190.0,120.0,6.5,50.0,90.0,150.0,210.0,22.0,0.0,NaN,50.0,0.0,0.0,75.0,0.0,0.0,0.0
7,75.0,75.0,1,1,False,190.0,120.0,6.5,False,inf,50.0,90.0,150.0,210.0,22.0,0.0,5,None,0,0.0,0.0,False,0.0,0.0,50.0,1,False,False,0,0,False,False,None,None,None,False,0,False,False,55.327749,55.327749,0,0,0.0,0,0,7,190.0,120.0,6.5,50.0,90.0,150.0,210.0,22.0,0.0,None,50.0,0,0.0,75.0,0.0,0.0,0,190.0,120.0,6.5,50.0,90.0,150.0,210.0,22.0,0.0,NaN,50.0,0.0,0.0,75.0,0.0,0.0,0.0
8,75.0,75.0,1,1,False,190.0,120.0,6.5,False,inf,50.0,90.0,150.0,210.0,22.0,0.0,5,None,0,0.0,0.0,False,0.0,0.0,50.0,1,False,False,0,0,False,Fals

In [53]:
for i in range(100):
    test.testSprintStrategyAtPopLevelTwoWave()
i

/Users/deligkaris.1/Library/CloudStorage/OneDrive-TheOhioStateUniversityWexnerMedicalCenter/MICROSIM/CODE/microsim/microsim/population.py:342: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  totalBPMedsAddedCapped.loc[totalBPMedsAddedCapped >= BaseTreatmentStrategy.MAX_BP_MEDS] = BaseTreatmentStrategy.MAX_BP_MEDS
/Users/deligkaris.1/Library/CloudStorage/OneDrive-TheOhioStateUniversityWexnerMedicalCenter/MICROSIM/CODE/microsim/microsim/gfr_equation.py:82: RuntimeWarning: divide by zero encountered in double_scalars
  return constant * (x.creatinine / crThreshold) ** exponent * 0.993**x.age
/Users/deligkaris.1/Library/CloudStorage/OneDrive-TheOhioStateUniversityWexnerMedicalCenter/MICROSIM/CODE/microsim/microsim/population.py:342: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice fr

/Users/deligkaris.1/Library/CloudStorage/OneDrive-TheOhioStateUniversityWexnerMedicalCenter/MICROSIM/CODE/microsim/microsim/population.py:342: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  totalBPMedsAddedCapped.loc[totalBPMedsAddedCapped >= BaseTreatmentStrategy.MAX_BP_MEDS] = BaseTreatmentStrategy.MAX_BP_MEDS
/Users/deligkaris.1/Library/CloudStorage/OneDrive-TheOhioStateUniversityWexnerMedicalCenter/MICROSIM/CODE/microsim/microsim/gfr_equation.py:82: RuntimeWarning: divide by zero encountered in double_scalars
  return constant * (x.creatinine / crThreshold) ** exponent * 0.993**x.age
/Users/deligkaris.1/Library/CloudStorage/OneDrive-TheOhioStateUniversityWexnerMedicalCenter/MICROSIM/CODE/microsim/microsim/population.py:342: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice fr

/Users/deligkaris.1/Library/CloudStorage/OneDrive-TheOhioStateUniversityWexnerMedicalCenter/MICROSIM/CODE/microsim/microsim/population.py:342: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  totalBPMedsAddedCapped.loc[totalBPMedsAddedCapped >= BaseTreatmentStrategy.MAX_BP_MEDS] = BaseTreatmentStrategy.MAX_BP_MEDS
/Users/deligkaris.1/Library/CloudStorage/OneDrive-TheOhioStateUniversityWexnerMedicalCenter/MICROSIM/CODE/microsim/microsim/gfr_equation.py:82: RuntimeWarning: divide by zero encountered in double_scalars
  return constant * (x.creatinine / crThreshold) ** exponent * 0.993**x.age
/Users/deligkaris.1/Library/CloudStorage/OneDrive-TheOhioStateUniversityWexnerMedicalCenter/MICROSIM/CODE/microsim/microsim/population.py:342: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice fr

/Users/deligkaris.1/Library/CloudStorage/OneDrive-TheOhioStateUniversityWexnerMedicalCenter/MICROSIM/CODE/microsim/microsim/population.py:342: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  totalBPMedsAddedCapped.loc[totalBPMedsAddedCapped >= BaseTreatmentStrategy.MAX_BP_MEDS] = BaseTreatmentStrategy.MAX_BP_MEDS
/Users/deligkaris.1/Library/CloudStorage/OneDrive-TheOhioStateUniversityWexnerMedicalCenter/MICROSIM/CODE/microsim/microsim/gfr_equation.py:82: RuntimeWarning: divide by zero encountered in double_scalars
  return constant * (x.creatinine / crThreshold) ** exponent * 0.993**x.age
/Users/deligkaris.1/Library/CloudStorage/OneDrive-TheOhioStateUniversityWexnerMedicalCenter/MICROSIM/CODE/microsim/microsim/population.py:342: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice fr

/Users/deligkaris.1/Library/CloudStorage/OneDrive-TheOhioStateUniversityWexnerMedicalCenter/MICROSIM/CODE/microsim/microsim/population.py:342: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  totalBPMedsAddedCapped.loc[totalBPMedsAddedCapped >= BaseTreatmentStrategy.MAX_BP_MEDS] = BaseTreatmentStrategy.MAX_BP_MEDS
/Users/deligkaris.1/Library/CloudStorage/OneDrive-TheOhioStateUniversityWexnerMedicalCenter/MICROSIM/CODE/microsim/microsim/gfr_equation.py:82: RuntimeWarning: divide by zero encountered in double_scalars
  return constant * (x.creatinine / crThreshold) ** exponent * 0.993**x.age
/Users/deligkaris.1/Library/CloudStorage/OneDrive-TheOhioStateUniversityWexnerMedicalCenter/MICROSIM/CODE/microsim/microsim/population.py:342: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice fr

...

/Users/deligkaris.1/Library/CloudStorage/OneDrive-TheOhioStateUniversityWexnerMedicalCenter/MICROSIM/CODE/microsim/microsim/population.py:342: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  totalBPMedsAddedCapped.loc[totalBPMedsAddedCapped >= BaseTreatmentStrategy.MAX_BP_MEDS] = BaseTreatmentStrategy.MAX_BP_MEDS
/Users/deligkaris.1/Library/CloudStorage/OneDrive-TheOhioStateUniversityWexnerMedicalCenter/MICROSIM/CODE/microsim/microsim/gfr_equation.py:82: RuntimeWarning: divide by zero encountered in double_scalars
  return constant * (x.creatinine / crThreshold) ** exponent * 0.993**x.age
/Users/deligkaris.1/Library/CloudStorage/OneDrive-TheOhioStateUniversityWexnerMedicalCenter/MICROSIM/CODE/microsim/microsim/population.py:342: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice fr

/Users/deligkaris.1/Library/CloudStorage/OneDrive-TheOhioStateUniversityWexnerMedicalCenter/MICROSIM/CODE/microsim/microsim/population.py:342: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  totalBPMedsAddedCapped.loc[totalBPMedsAddedCapped >= BaseTreatmentStrategy.MAX_BP_MEDS] = BaseTreatmentStrategy.MAX_BP_MEDS
/Users/deligkaris.1/Library/CloudStorage/OneDrive-TheOhioStateUniversityWexnerMedicalCenter/MICROSIM/CODE/microsim/microsim/gfr_equation.py:82: RuntimeWarning: divide by zero encountered in double_scalars
  return constant * (x.creatinine / crThreshold) ** exponent * 0.993**x.age
/Users/deligkaris.1/Library/CloudStorage/OneDrive-TheOhioStateUniversityWexnerMedicalCenter/MICROSIM/CODE/microsim/microsim/population.py:342: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice fr

/Users/deligkaris.1/Library/CloudStorage/OneDrive-TheOhioStateUniversityWexnerMedicalCenter/MICROSIM/CODE/microsim/microsim/population.py:342: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  totalBPMedsAddedCapped.loc[totalBPMedsAddedCapped >= BaseTreatmentStrategy.MAX_BP_MEDS] = BaseTreatmentStrategy.MAX_BP_MEDS
/Users/deligkaris.1/Library/CloudStorage/OneDrive-TheOhioStateUniversityWexnerMedicalCenter/MICROSIM/CODE/microsim/microsim/gfr_equation.py:82: RuntimeWarning: divide by zero encountered in double_scalars
  return constant * (x.creatinine / crThreshold) ** exponent * 0.993**x.age
/Users/deligkaris.1/Library/CloudStorage/OneDrive-TheOhioStateUniversityWexnerMedicalCenter/MICROSIM/CODE/microsim/microsim/population.py:342: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice fr

/Users/deligkaris.1/Library/CloudStorage/OneDrive-TheOhioStateUniversityWexnerMedicalCenter/MICROSIM/CODE/microsim/microsim/population.py:342: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  totalBPMedsAddedCapped.loc[totalBPMedsAddedCapped >= BaseTreatmentStrategy.MAX_BP_MEDS] = BaseTreatmentStrategy.MAX_BP_MEDS
/Users/deligkaris.1/Library/CloudStorage/OneDrive-TheOhioStateUniversityWexnerMedicalCenter/MICROSIM/CODE/microsim/microsim/gfr_equation.py:82: RuntimeWarning: divide by zero encountered in double_scalars
  return constant * (x.creatinine / crThreshold) ** exponent * 0.993**x.age
/Users/deligkaris.1/Library/CloudStorage/OneDrive-TheOhioStateUniversityWexnerMedicalCenter/MICROSIM/CODE/microsim/microsim/population.py:342: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice fr

99

In [56]:
for i in range(100):
    test.testSprintStrategyAtPopLevelOneWave()
i

/Users/deligkaris.1/Library/CloudStorage/OneDrive-TheOhioStateUniversityWexnerMedicalCenter/MICROSIM/CODE/microsim/microsim/population.py:342: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  totalBPMedsAddedCapped.loc[totalBPMedsAddedCapped >= BaseTreatmentStrategy.MAX_BP_MEDS] = BaseTreatmentStrategy.MAX_BP_MEDS
/Users/deligkaris.1/Library/CloudStorage/OneDrive-TheOhioStateUniversityWexnerMedicalCenter/MICROSIM/CODE/microsim/microsim/gfr_equation.py:82: RuntimeWarning: divide by zero encountered in double_scalars
  return constant * (x.creatinine / crThreshold) ** exponent * 0.993**x.age
/Users/deligkaris.1/Library/CloudStorage/OneDrive-TheOhioStateUniversityWexnerMedicalCenter/MICROSIM/CODE/microsim/microsim/population.py:342: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice fr

/Users/deligkaris.1/Library/CloudStorage/OneDrive-TheOhioStateUniversityWexnerMedicalCenter/MICROSIM/CODE/microsim/microsim/population.py:342: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  totalBPMedsAddedCapped.loc[totalBPMedsAddedCapped >= BaseTreatmentStrategy.MAX_BP_MEDS] = BaseTreatmentStrategy.MAX_BP_MEDS
/Users/deligkaris.1/Library/CloudStorage/OneDrive-TheOhioStateUniversityWexnerMedicalCenter/MICROSIM/CODE/microsim/microsim/gfr_equation.py:82: RuntimeWarning: divide by zero encountered in double_scalars
  return constant * (x.creatinine / crThreshold) ** exponent * 0.993**x.age
/Users/deligkaris.1/Library/CloudStorage/OneDrive-TheOhioStateUniversityWexnerMedicalCenter/MICROSIM/CODE/microsim/microsim/population.py:342: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice fr

/Users/deligkaris.1/Library/CloudStorage/OneDrive-TheOhioStateUniversityWexnerMedicalCenter/MICROSIM/CODE/microsim/microsim/population.py:342: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  totalBPMedsAddedCapped.loc[totalBPMedsAddedCapped >= BaseTreatmentStrategy.MAX_BP_MEDS] = BaseTreatmentStrategy.MAX_BP_MEDS
/Users/deligkaris.1/Library/CloudStorage/OneDrive-TheOhioStateUniversityWexnerMedicalCenter/MICROSIM/CODE/microsim/microsim/gfr_equation.py:82: RuntimeWarning: divide by zero encountered in double_scalars
  return constant * (x.creatinine / crThreshold) ** exponent * 0.993**x.age
/Users/deligkaris.1/Library/CloudStorage/OneDrive-TheOhioStateUniversityWexnerMedicalCenter/MICROSIM/CODE/microsim/microsim/population.py:342: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice fr

/Users/deligkaris.1/Library/CloudStorage/OneDrive-TheOhioStateUniversityWexnerMedicalCenter/MICROSIM/CODE/microsim/microsim/population.py:342: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  totalBPMedsAddedCapped.loc[totalBPMedsAddedCapped >= BaseTreatmentStrategy.MAX_BP_MEDS] = BaseTreatmentStrategy.MAX_BP_MEDS
/Users/deligkaris.1/Library/CloudStorage/OneDrive-TheOhioStateUniversityWexnerMedicalCenter/MICROSIM/CODE/microsim/microsim/gfr_equation.py:82: RuntimeWarning: divide by zero encountered in double_scalars
  return constant * (x.creatinine / crThreshold) ** exponent * 0.993**x.age
/Users/deligkaris.1/Library/CloudStorage/OneDrive-TheOhioStateUniversityWexnerMedicalCenter/MICROSIM/CODE/microsim/microsim/population.py:342: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice fr

/Users/deligkaris.1/Library/CloudStorage/OneDrive-TheOhioStateUniversityWexnerMedicalCenter/MICROSIM/CODE/microsim/microsim/population.py:342: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  totalBPMedsAddedCapped.loc[totalBPMedsAddedCapped >= BaseTreatmentStrategy.MAX_BP_MEDS] = BaseTreatmentStrategy.MAX_BP_MEDS
/Users/deligkaris.1/Library/CloudStorage/OneDrive-TheOhioStateUniversityWexnerMedicalCenter/MICROSIM/CODE/microsim/microsim/gfr_equation.py:82: RuntimeWarning: divide by zero encountered in double_scalars
  return constant * (x.creatinine / crThreshold) ** exponent * 0.993**x.age
/Users/deligkaris.1/Library/CloudStorage/OneDrive-TheOhioStateUniversityWexnerMedicalCenter/MICROSIM/CODE/microsim/microsim/population.py:342: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice fr

...

/Users/deligkaris.1/Library/CloudStorage/OneDrive-TheOhioStateUniversityWexnerMedicalCenter/MICROSIM/CODE/microsim/microsim/population.py:342: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  totalBPMedsAddedCapped.loc[totalBPMedsAddedCapped >= BaseTreatmentStrategy.MAX_BP_MEDS] = BaseTreatmentStrategy.MAX_BP_MEDS
/Users/deligkaris.1/Library/CloudStorage/OneDrive-TheOhioStateUniversityWexnerMedicalCenter/MICROSIM/CODE/microsim/microsim/gfr_equation.py:82: RuntimeWarning: divide by zero encountered in double_scalars
  return constant * (x.creatinine / crThreshold) ** exponent * 0.993**x.age
/Users/deligkaris.1/Library/CloudStorage/OneDrive-TheOhioStateUniversityWexnerMedicalCenter/MICROSIM/CODE/microsim/microsim/population.py:342: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice fr

/Users/deligkaris.1/Library/CloudStorage/OneDrive-TheOhioStateUniversityWexnerMedicalCenter/MICROSIM/CODE/microsim/microsim/population.py:342: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  totalBPMedsAddedCapped.loc[totalBPMedsAddedCapped >= BaseTreatmentStrategy.MAX_BP_MEDS] = BaseTreatmentStrategy.MAX_BP_MEDS
/Users/deligkaris.1/Library/CloudStorage/OneDrive-TheOhioStateUniversityWexnerMedicalCenter/MICROSIM/CODE/microsim/microsim/gfr_equation.py:82: RuntimeWarning: divide by zero encountered in double_scalars
  return constant * (x.creatinine / crThreshold) ** exponent * 0.993**x.age
/Users/deligkaris.1/Library/CloudStorage/OneDrive-TheOhioStateUniversityWexnerMedicalCenter/MICROSIM/CODE/microsim/microsim/population.py:342: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice fr

/Users/deligkaris.1/Library/CloudStorage/OneDrive-TheOhioStateUniversityWexnerMedicalCenter/MICROSIM/CODE/microsim/microsim/population.py:342: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  totalBPMedsAddedCapped.loc[totalBPMedsAddedCapped >= BaseTreatmentStrategy.MAX_BP_MEDS] = BaseTreatmentStrategy.MAX_BP_MEDS
/Users/deligkaris.1/Library/CloudStorage/OneDrive-TheOhioStateUniversityWexnerMedicalCenter/MICROSIM/CODE/microsim/microsim/gfr_equation.py:82: RuntimeWarning: divide by zero encountered in double_scalars
  return constant * (x.creatinine / crThreshold) ** exponent * 0.993**x.age
/Users/deligkaris.1/Library/CloudStorage/OneDrive-TheOhioStateUniversityWexnerMedicalCenter/MICROSIM/CODE/microsim/microsim/population.py:342: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice fr

/Users/deligkaris.1/Library/CloudStorage/OneDrive-TheOhioStateUniversityWexnerMedicalCenter/MICROSIM/CODE/microsim/microsim/population.py:342: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  totalBPMedsAddedCapped.loc[totalBPMedsAddedCapped >= BaseTreatmentStrategy.MAX_BP_MEDS] = BaseTreatmentStrategy.MAX_BP_MEDS
/Users/deligkaris.1/Library/CloudStorage/OneDrive-TheOhioStateUniversityWexnerMedicalCenter/MICROSIM/CODE/microsim/microsim/gfr_equation.py:82: RuntimeWarning: divide by zero encountered in double_scalars
  return constant * (x.creatinine / crThreshold) ** exponent * 0.993**x.age


99

In [59]:
for i in range(100):
    test.testSprintStrategyAtPopLevelFiveWave()
i

/Users/deligkaris.1/Library/CloudStorage/OneDrive-TheOhioStateUniversityWexnerMedicalCenter/MICROSIM/CODE/microsim/microsim/population.py:342: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  totalBPMedsAddedCapped.loc[totalBPMedsAddedCapped >= BaseTreatmentStrategy.MAX_BP_MEDS] = BaseTreatmentStrategy.MAX_BP_MEDS
/Users/deligkaris.1/Library/CloudStorage/OneDrive-TheOhioStateUniversityWexnerMedicalCenter/MICROSIM/CODE/microsim/microsim/gfr_equation.py:82: RuntimeWarning: divide by zero encountered in double_scalars
  return constant * (x.creatinine / crThreshold) ** exponent * 0.993**x.age
/Users/deligkaris.1/Library/CloudStorage/OneDrive-TheOhioStateUniversityWexnerMedicalCenter/MICROSIM/CODE/microsim/microsim/population.py:342: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice fr

/Users/deligkaris.1/Library/CloudStorage/OneDrive-TheOhioStateUniversityWexnerMedicalCenter/MICROSIM/CODE/microsim/microsim/gfr_equation.py:82: RuntimeWarning: divide by zero encountered in double_scalars
  return constant * (x.creatinine / crThreshold) ** exponent * 0.993**x.age
/Users/deligkaris.1/Library/CloudStorage/OneDrive-TheOhioStateUniversityWexnerMedicalCenter/MICROSIM/CODE/microsim/microsim/population.py:342: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  totalBPMedsAddedCapped.loc[totalBPMedsAddedCapped >= BaseTreatmentStrategy.MAX_BP_MEDS] = BaseTreatmentStrategy.MAX_BP_MEDS
/Users/deligkaris.1/Library/CloudStorage/OneDrive-TheOhioStateUniversityWexnerMedicalCenter/MICROSIM/CODE/microsim/microsim/gfr_equation.py:82: RuntimeWarning: divide by zero encountered in double_scalars
  return const

/Users/deligkaris.1/Library/CloudStorage/OneDrive-TheOhioStateUniversityWexnerMedicalCenter/MICROSIM/CODE/microsim/microsim/population.py:342: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  totalBPMedsAddedCapped.loc[totalBPMedsAddedCapped >= BaseTreatmentStrategy.MAX_BP_MEDS] = BaseTreatmentStrategy.MAX_BP_MEDS
/Users/deligkaris.1/Library/CloudStorage/OneDrive-TheOhioStateUniversityWexnerMedicalCenter/MICROSIM/CODE/microsim/microsim/gfr_equation.py:82: RuntimeWarning: divide by zero encountered in double_scalars
  return constant * (x.creatinine / crThreshold) ** exponent * 0.993**x.age
/Users/deligkaris.1/Library/CloudStorage/OneDrive-TheOhioStateUniversityWexnerMedicalCenter/MICROSIM/CODE/microsim/microsim/population.py:342: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice fr

/Users/deligkaris.1/Library/CloudStorage/OneDrive-TheOhioStateUniversityWexnerMedicalCenter/MICROSIM/CODE/microsim/microsim/population.py:342: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  totalBPMedsAddedCapped.loc[totalBPMedsAddedCapped >= BaseTreatmentStrategy.MAX_BP_MEDS] = BaseTreatmentStrategy.MAX_BP_MEDS
/Users/deligkaris.1/Library/CloudStorage/OneDrive-TheOhioStateUniversityWexnerMedicalCenter/MICROSIM/CODE/microsim/microsim/gfr_equation.py:82: RuntimeWarning: divide by zero encountered in double_scalars
  return constant * (x.creatinine / crThreshold) ** exponent * 0.993**x.age
/Users/deligkaris.1/Library/CloudStorage/OneDrive-TheOhioStateUniversityWexnerMedicalCenter/MICROSIM/CODE/microsim/microsim/population.py:342: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice fr

/Users/deligkaris.1/Library/CloudStorage/OneDrive-TheOhioStateUniversityWexnerMedicalCenter/MICROSIM/CODE/microsim/microsim/population.py:342: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  totalBPMedsAddedCapped.loc[totalBPMedsAddedCapped >= BaseTreatmentStrategy.MAX_BP_MEDS] = BaseTreatmentStrategy.MAX_BP_MEDS
/Users/deligkaris.1/Library/CloudStorage/OneDrive-TheOhioStateUniversityWexnerMedicalCenter/MICROSIM/CODE/microsim/microsim/gfr_equation.py:82: RuntimeWarning: divide by zero encountered in double_scalars
  return constant * (x.creatinine / crThreshold) ** exponent * 0.993**x.age
/Users/deligkaris.1/Library/CloudStorage/OneDrive-TheOhioStateUniversityWexnerMedicalCenter/MICROSIM/CODE/microsim/microsim/population.py:342: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice fr

...

/Users/deligkaris.1/Library/CloudStorage/OneDrive-TheOhioStateUniversityWexnerMedicalCenter/MICROSIM/CODE/microsim/microsim/population.py:342: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  totalBPMedsAddedCapped.loc[totalBPMedsAddedCapped >= BaseTreatmentStrategy.MAX_BP_MEDS] = BaseTreatmentStrategy.MAX_BP_MEDS
/Users/deligkaris.1/Library/CloudStorage/OneDrive-TheOhioStateUniversityWexnerMedicalCenter/MICROSIM/CODE/microsim/microsim/gfr_equation.py:82: RuntimeWarning: divide by zero encountered in double_scalars
  return constant * (x.creatinine / crThreshold) ** exponent * 0.993**x.age
/Users/deligkaris.1/Library/CloudStorage/OneDrive-TheOhioStateUniversityWexnerMedicalCenter/MICROSIM/CODE/microsim/microsim/population.py:342: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice fr

/Users/deligkaris.1/Library/CloudStorage/OneDrive-TheOhioStateUniversityWexnerMedicalCenter/MICROSIM/CODE/microsim/microsim/population.py:342: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  totalBPMedsAddedCapped.loc[totalBPMedsAddedCapped >= BaseTreatmentStrategy.MAX_BP_MEDS] = BaseTreatmentStrategy.MAX_BP_MEDS
/Users/deligkaris.1/Library/CloudStorage/OneDrive-TheOhioStateUniversityWexnerMedicalCenter/MICROSIM/CODE/microsim/microsim/gfr_equation.py:82: RuntimeWarning: divide by zero encountered in double_scalars
  return constant * (x.creatinine / crThreshold) ** exponent * 0.993**x.age
/Users/deligkaris.1/Library/CloudStorage/OneDrive-TheOhioStateUniversityWexnerMedicalCenter/MICROSIM/CODE/microsim/microsim/population.py:342: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice fr

/Users/deligkaris.1/Library/CloudStorage/OneDrive-TheOhioStateUniversityWexnerMedicalCenter/MICROSIM/CODE/microsim/microsim/population.py:342: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  totalBPMedsAddedCapped.loc[totalBPMedsAddedCapped >= BaseTreatmentStrategy.MAX_BP_MEDS] = BaseTreatmentStrategy.MAX_BP_MEDS
/Users/deligkaris.1/Library/CloudStorage/OneDrive-TheOhioStateUniversityWexnerMedicalCenter/MICROSIM/CODE/microsim/microsim/gfr_equation.py:82: RuntimeWarning: divide by zero encountered in double_scalars
  return constant * (x.creatinine / crThreshold) ** exponent * 0.993**x.age
/Users/deligkaris.1/Library/CloudStorage/OneDrive-TheOhioStateUniversityWexnerMedicalCenter/MICROSIM/CODE/microsim/microsim/population.py:342: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice fr

/Users/deligkaris.1/Library/CloudStorage/OneDrive-TheOhioStateUniversityWexnerMedicalCenter/MICROSIM/CODE/microsim/microsim/population.py:342: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  totalBPMedsAddedCapped.loc[totalBPMedsAddedCapped >= BaseTreatmentStrategy.MAX_BP_MEDS] = BaseTreatmentStrategy.MAX_BP_MEDS
/Users/deligkaris.1/Library/CloudStorage/OneDrive-TheOhioStateUniversityWexnerMedicalCenter/MICROSIM/CODE/microsim/microsim/gfr_equation.py:82: RuntimeWarning: divide by zero encountered in double_scalars
  return constant * (x.creatinine / crThreshold) ** exponent * 0.993**x.age
/Users/deligkaris.1/Library/CloudStorage/OneDrive-TheOhioStateUniversityWexnerMedicalCenter/MICROSIM/CODE/microsim/microsim/population.py:342: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice fr

99